# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.6000,73.0833,27.21,77,71,1.73,MV,1700949064
1,1,undurkhaan,47.3194,110.6556,-12.89,89,100,2.26,MN,1700949064
2,2,songea,-10.6833,35.6500,19.30,85,100,1.99,TZ,1700949064
3,3,grytviken,-54.2811,-36.5092,0.34,77,23,4.14,GS,1700949064
4,4,newman,37.3138,-121.0208,16.80,38,0,2.68,US,1700949065


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 19) & (city_data_df["Max Temp"] < 28) & 
                                (city_data_df["Wind Speed"] > 2) & (city_data_df["Wind Speed"] < 10) & 
                                 (city_data_df["Humidity"] > 30) & (city_data_df["Humidity"] < 60), :]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna(how='any')

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,medina,24.4686,39.6142,26.24,47,0,3.09,SA,1700948967
53,53,pimentel,-6.8367,-79.9342,25.16,57,0,9.26,PE,1700949075
92,92,oodweyne,9.4092,45.0640,21.02,43,91,3.59,SO,1700949085
120,120,alice springs,-23.7000,133.8833,25.75,47,100,3.09,AU,1700949092
165,165,salinas,36.6777,-121.6555,19.24,38,0,5.14,US,1700949101
195,195,trevelin,-43.0858,-71.4639,20.71,46,97,3.81,AR,1700949107
268,268,lompoc,34.6391,-120.4579,20.56,49,0,4.47,US,1700949122
282,282,arraial do cabo,-22.9661,-42.0278,22.92,53,75,3.60,BR,1700949126
366,366,puerto deseado,-47.7503,-65.8938,21.63,31,69,4.34,AR,1700949145
371,371,kerikeri,-35.2268,173.9474,19.47,54,2,3.24,NZ,1700949146


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/4v/cb_1rxpn3hgb9k0psp8120wm0000gn/T/ipykernel_98975/1672974955.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
22,medina,SA,24.4686,39.6142,47,
53,pimentel,PE,-6.8367,-79.9342,57,
92,oodweyne,SO,9.4092,45.0640,43,
120,alice springs,AU,-23.7000,133.8833,47,
165,salinas,US,36.6777,-121.6555,38,
195,trevelin,AR,-43.0858,-71.4639,46,
268,lompoc,US,34.6391,-120.4579,49,
282,arraial do cabo,BR,-22.9661,-42.0278,53,
366,puerto deseado,AR,-47.7503,-65.8938,31,
371,kerikeri,NZ,-35.2268,173.9474,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
medina - nearest hotel: The Oberoi Madina
pimentel - nearest hotel: Hospedaje Calle del Sol
oodweyne - nearest hotel: فندق المدينه
alice springs - nearest hotel: Aurora Alice Springs
salinas - nearest hotel: The Traveler's Hotel
trevelin - nearest hotel: Casa de Piedra
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
arraial do cabo - nearest hotel: No hotel found
puerto deseado - nearest hotel: Los Olmos
kerikeri - nearest hotel: Avalon Resort
san diego - nearest hotel: ANdAZ San Diego
bardai - nearest hotel: No hotel found
yanchep - nearest hotel: Yanchep Inn
villa gesell - nearest hotel: Hotel El Faro
port said - nearest hotel: The New Continental Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
22,medina,SA,24.4686,39.6142,47,The Oberoi Madina
53,pimentel,PE,-6.8367,-79.9342,57,Hospedaje Calle del Sol
92,oodweyne,SO,9.4092,45.0640,43,فندق المدينه
120,alice springs,AU,-23.7000,133.8833,47,Aurora Alice Springs
165,salinas,US,36.6777,-121.6555,38,The Traveler's Hotel
195,trevelin,AR,-43.0858,-71.4639,46,Casa de Piedra
268,lompoc,US,34.6391,-120.4579,49,Embassy Suites by Hilton Lompoc Central Coast
282,arraial do cabo,BR,-22.9661,-42.0278,53,No hotel found
366,puerto deseado,AR,-47.7503,-65.8938,31,Los Olmos
371,kerikeri,NZ,-35.2268,173.9474,54,Avalon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    color = "City", 
    hover_cols=["Country", "Hotel Name"]
)

# Display the map plot_1
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)